# Build a training dataset

In [12]:
from feast import FeatureStore
import pandas as pd
from datetime import datetime

entity_df = pd.DataFrame.from_dict({
    "driver_id": [1001, 1002, 1003, 1004],
    "event_timestamp": [
        datetime(2021, 4, 12, 10, 59, 42),
        datetime(2021, 4, 12, 8,  12, 10),
        datetime(2021, 4, 12, 16, 40, 26),
        datetime(2021, 4, 12, 15, 1 , 12)
    ]
})

feast_repo_path = "/Users/zrrjdd/software/github/langchain-learn/myfeature_repo/feature_repo/"
store = FeatureStore(repo_path=feast_repo_path)

training_df = store.get_historical_features(
    entity_df=entity_df,
    features = [
        'driver_hourly_stats:conv_rate',
        'driver_hourly_stats:acc_rate',
        'driver_hourly_stats:avg_daily_trips'
    ],
).to_df()

print(training_df.head())

# Train model
# model = ml.fit(training_df)

   driver_id           event_timestamp  conv_rate  acc_rate  avg_daily_trips
0       1001 2021-04-12 10:59:42+00:00   0.769165  0.467336              690
1       1002 2021-04-12 08:12:10+00:00   0.565716  0.084002              212
2       1003 2021-04-12 16:40:26+00:00   0.283134  0.544876              997
3       1004 2021-04-12 15:01:12+00:00   0.532676  0.634407              830


# 使用

In [5]:
from feast import FeatureStore
from langchain.prompts import PromptTemplate, StringPromptTemplate

In [4]:
feast_repo_path = "/Users/zrrjdd/software/github/langchain-learn/myfeature_repo/feature_repo/"
store = FeatureStore(repo_path=feast_repo_path)

In [13]:
template = """Given the driver's up to date stats, write them note relaying those stats to them.
If they have a conversation rate above .5, give them a compliment. Otherwise, make a silly joke about chickens at the end to make them feel better

Here are the drivers stats:
Conversation rate: {conv_rate}
Acceptance rate: {acc_rate}
Average Daily Trips: {avg_daily_trips}

Your response:"""
prompt = PromptTemplate.from_template(template)

In [17]:
class FeastPromptTemplate(StringPromptTemplate):
    
    def format(self, **kwargs) -> str:
        driver_id = kwargs.pop("driver_id")
        feature_vector = store.get_online_features(
            features=[
                'driver_hourly_stats:conv_rate',
                'driver_hourly_stats:acc_rate',
                'driver_hourly_stats:avg_daily_trips'
            ],
            entity_rows=[{"driver_id": driver_id}]
        ).to_dict()
        print(feature_vector)
        kwargs["conv_rate"] = feature_vector["conv_rate"][0]
        kwargs["acc_rate"] = feature_vector["acc_rate"][0]
        kwargs["avg_daily_trips"] = feature_vector["avg_daily_trips"][0]
        return prompt.format(**kwargs)

In [18]:
prompt_template = FeastPromptTemplate(input_variables=["driver_id"])

In [19]:
print(prompt_template.format(driver_id=1001))

{'driver_id': [1001], 'acc_rate': [None], 'conv_rate': [None], 'avg_daily_trips': [None]}
Given the driver's up to date stats, write them note relaying those stats to them.
If they have a conversation rate above .5, give them a compliment. Otherwise, make a silly joke about chickens at the end to make them feel better

Here are the drivers stats:
Conversation rate: None
Acceptance rate: None
Average Daily Trips: None

Your response:


# 创建自定义提示模版

In [20]:
import inspect

def get_source_code(function_name):
    # Get the source code of the function
    return inspect.getsource(function_name)


In [21]:
from langchain.prompts import StringPromptTemplate
from pydantic import BaseModel, validator


class FunctionExplainerPromptTemplate(StringPromptTemplate, BaseModel):
    """ A custom prompt template that takes in the function name as input, and formats the prompt template to provide the source code of the function. """

    @validator("input_variables")
    def validate_input_variables(cls, v):
        """ Validate that the input variables are correct. """
        if len(v) != 1 or "function_name" not in v:
            raise ValueError("function_name must be the only input_variable.")
        return v

    def format(self, **kwargs) -> str:
        # Get the source code of the function
        source_code = get_source_code(kwargs["function_name"])

        # Generate the prompt to be sent to the language model
        prompt = f"""
        Given the function name and source code, generate an English language explanation of the function.
        Function Name: {kwargs["function_name"].__name__}
        Source Code:
        {source_code}
        Explanation:
        """
        return prompt
    
    def _prompt_type(self):
        return "function-explainer"

In [22]:
fn_explainer = FunctionExplainerPromptTemplate(input_variables=["function_name"])

# Generate a prompt for the function "get_source_code"
prompt = fn_explainer.format(function_name=get_source_code)
print(prompt)


        Given the function name and source code, generate an English language explanation of the function.
        Function Name: get_source_code
        Source Code:
        def get_source_code(function_name):
    # Get the source code of the function
    return inspect.getsource(function_name)

        Explanation:
        


In [23]:
from langchain.llms import OpenAI

In [24]:
llm = OpenAI(temperature=0)

In [26]:
r = llm(prompt=prompt)
print(r)


This function takes in a function name as an argument and returns the source code of that function. It does this by using the inspect module to get the source code of the given function.
